In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Oppgave 1
## Hvor mange unike IP-adresser?

In [66]:
df = pd.read_csv("O1-Flow.csv", delimiter=";")

SOURCE og DEST inneholder IP-adresse med portnummer. Vi er kun interessert i IP-adresser.
Ekstraherer kun IP-adressene til nye kolonner, og deretter teller unike verdier

In [67]:
src_ips = df['SOURCE'].str.split(":").str[0].rename("SRC_IP")
dst_ips = df['DEST'].str.split(":").str[0].rename("DST_IP")
src_ports = df['SOURCE'].str.split(":").str[1].rename("SRC_PORT")
dst_ports = df['DEST'].str.split(":").str[1].rename("DST_PORT")

src_dst = pd.concat([src_ips, dst_ips], axis = 1)
n_unq_ips = src_dst[["SRC_IP", "DST_IP"]].nunique()
n_unq_ips

SRC_IP     4
DST_IP    54
dtype: int64

# Oppgave 2
## Hva er den total datamengde per IP, per retning?

Før jeg summerer datamengdene, tilføyer jeg de nyopprettede kolonnene fra forrige oppgave i nåværende dataframe.
Dette gir dataframe-en noe redundant info, men nå kan vi gruppere kun med IP

In [68]:
new_df = pd.concat([df, src_ips, src_ports, dst_ips, dst_ports], axis=1)
total_data = new_df.groupby(["SRC_IP", "DST_IP"])['BYTES'].sum()


In [69]:
total_data

SRC_IP        DST_IP       
10.23.35.133  8.8.8.8            5305
10.23.35.60   8.8.8.8             980
10.23.35.61   8.8.8.8           74604
8.8.8.8       10.13.232.147      2546
              10.13.232.190       742
              10.13.232.218      2974
              10.13.232.33        164
              10.13.232.42        255
              10.13.232.45       3007
              10.13.232.50       1405
              10.13.232.57       3270
              10.13.232.83       1662
              10.13.232.88        462
              10.13.232.89        877
              10.13.232.90        371
              10.13.232.91        164
              10.13.232.97         92
              10.23.35.133       8920
              10.23.35.60        2287
              10.23.35.61      220827
              10.41.164.54         92
              10.41.165.1        4903
              10.41.255.1         492
              10.41.255.14        208
              10.41.255.17        174
              10.41.25

# Oppgave 3
## Hva er prosentfordelingen av protokollene?

In [70]:
n_uniq_protocols = new_df['PROTO'].value_counts(normalize="True")
n_uniq_protocols

PROTO
udp     0.979
tcp     0.020
icmp    0.001
Name: proportion, dtype: float64

# Oppgave 4
## Hva er prosentfordelingen blant portene?
### (Alle porter større enn 1024 kan omtales som <i>high ports</i>)

Oppdaget at det er noen av verdiene i de opprinnelige SOURCE og DEST kolonnene ikke er oppført med portnummer. I slike tilfeller vil det oppstå NaN.

In [121]:
all_ports = pd.concat([new_df["SRC_PORT"], new_df["DST_PORT"]]).reset_index(drop=True).rename("PORTS")
all_ports = all_ports[all_ports.notna()].astype(int)
all_ports[all_ports > 1024] = "high port"
all_ports.value_counts(normalize="True").round(3)


/tmp/ipykernel_47921/2344748536.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'high port' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_ports[all_ports > 1024] = "high port"


PORTS
high port    0.499
53           0.498
853          0.002
1024         0.001
Name: proportion, dtype: float64